In [71]:
import numpy as np 
import timeit
import pandas as pd
import matplotlib.pyplot as plt
import logbin as lb
import logbinFixedSIze as lbx

plt.rcParams['figure.dpi'] = 300 # 200 e.g. is really fine, but slower

m_array = [2, 4, 8, 16, 32]
colours = ["blue", "orange", "green", "red", "black"]

nodes = 1000000
trials = 100

def LoadFreq(isBa, isEr, N, m, runs):

    if isBa:
        filename = "BA_N" + str(N) + "m" + str(m) + "R" + str(runs) + "FREQ.csv"
    elif isEr:
        filename = "ER_N" + str(N) + "m" + str(m) + "R" + str(runs) + "FREQ.csv"
    else:
        filename = "MIX_N" + str(N) + "m" + str(m) + "R" + str(runs) + "P23FREQ.csv"

    df = pd.read_csv(filename)
    df.columns = ["Node","k"]
    df = df.set_index("Node")
    #df = df.reindex(list(range(1,df.index.max()+1)),fill_value=0)

    return df

In [72]:

def Clean(nodes, trials, m, scale):

    full_freqs = LoadFreq(False, False, nodes, m, trials)

    k_max = full_freqs["k"].max()

    t_max = int(full_freqs["k"].idxmax()/nodes)
    bins, probs, binedges = lbx.oldbin(full_freqs["k"].iloc[nodes * t_max: nodes * (t_max+1)], scale=scale)

    binned_probs = []

    for i in range(trials):
        single_series = full_freqs["k"].iloc[nodes * i: nodes * (i+1)]
        centres, probs = lbx.logbin(single_series, binedges, scale=scale)
        binned_probs.append(probs)
    binned_probs = pd.DataFrame(binned_probs)

    means = binned_probs.mean()
    errs = binned_probs.sem()
    bins = pd.Series(bins)

    final_df = pd.concat([bins, means, errs], axis=1)
    final_df.columns = ["Centres", "Mean", "Sem"]

    return final_df

In [73]:
for i in m_array:
    print("Start" + str(i))
    data = Clean(nodes, trials, i, 1.1)
    print("Writing" + str(i))
    filename = "MIX_N" + str(nodes) + "m" + str(i) + "R" + str(trials) + "S" + str(scale) + "CLEAN.csv"
    data.to_csv(filename)
    print("Finish" + str(i))

Start2
Writing2
Finish2
Start4
Writing4
Finish4
Start8
Writing8
Finish8
Start16
Writing16
Finish16
Start32
Writing32
Finish32


In [74]:
#plt.plot(np.log10(np.arange(0,1000000)), full_freqs["k"].iloc[0:1000000])

